
# Practicum II - Movie Database Applicaton
### Collaborators: Abasamia Akpan, Rebecca Brent and Alessia Pizzoccheri

This Notebook provides context, documentation and sample code for a Database Application using IMBd datasets; for clarity, this document has been divided into the following sections:

### 1. Introduction
### 2. Assumptions
### 3. Logical Models
### 4. Database and Relations Definition¶
### 1. Introduction
#### 1.1. Looking at the data
In this section, we analyze and deconstruct the various data sets found on the IMBd page; data is organized in seven TSV files, specifically:

Title AKAs
Title Basics
Title Episodes
Title Ratings
Title Crew
Name Basics
Title Principles Our first step is to establish relationships between each data set and identify a primary key (PK) on each table. Since no information was provided regarding the role of columns, we devised a solution to find unique identifiers within each data set; that is, for each table we compare the total number of rows against the total number of unique values for a given column.
1.2 Establishing PKs
Following the system aforementioned, we identified the following primary keys

## Notes


#### title_basics is fine but wont let me add a null key at the end to store the foreign keys in the other table that dont map to a primary key in title_basics table so i commented it out.  Loads 99 except for that%

#### title ratings has foreign keys that dont exist . I tried to mask them and they worked before but table insertion failed today- #### We could make a decision if we want to discard foreign keys tconst, parentTconst and nconst that dont map to a primary key in title_basics or name_basics table and consider them as bad data

#### title episodes has the same issue as above

#### title AKA's has the same issue as above

#### name basics is fine

#### principals is fine 

#### all the other tables should be working

#### we should probably chage some lengths of some columns  or do this if performance is a concern
#### https://stackoverflow.com/questions/8746207/1071-specified-key-was-too-long-max-key-length-is-1000-bytes
Have no idea how to implement though
.....

In [2]:

import pandas as pd
import numpy as np
import os
import sys
import pip
pip.main(['install','prettytable'])
pip.main(['install','mysql-connector-python'])
pip.main(['install','tqdm'])
pip.main(['install','pymysql'])

from tqdm import tqdm



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [466]:
import mysql.connector
from prettytable import PrettyTable 
from prettytable import from_csv
from mysql.connector import Error

def create_connection_nodb(host_name, user_name, user_password):
    """ create a connection to mysql host
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_database(connection, db_name):
    """ create a database 
    :param connection: Connection object
    :param db_name: name of database in form of string
    :return:
    """
    cursor = connection.cursor()
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + db_name
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(host_name, user_name, user_password, db_name):
    """ create a connection to the (possibly newly created) database
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :param db_name: string name of database 
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        table_name = create_table_sql.split(" ")[5].split("(")[0]
        print("Successfully created table: " + table_name)
    except Error as e:
        print(e)
        
def main():
    
    database = "imbd_application"
    host_name = "localhost"
    user_name = "root"
    user_password = "Odyssey22"
    
    create_table_titlebasics = """CREATE TABLE IF NOT EXISTS titlebasics(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    titleType TEXT,
                                    primaryTitle TEXT,
                                    originalTitle TEXT,
                                    isAdult BOOLEAN,
                                    startYear DATE,
                                    endYear DATE,
                                    runTimeMinutes TIME)"""
    
    create_table_titleratings = """CREATE TABLE IF NOT EXISTS titleratings(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    averageRating FLOAT,
                                    numVotes INT,
                                    FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""


    create_table_titleepisodes = """CREATE TABLE IF NOT EXISTS titleepisodes(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    seasonNumber INT,
                                    episodeNumber INT,
                                    parentTconst CHAR(9),
                                    FOREIGN KEY (parentTconst) REFERENCES titlebasics(tconst))"""

    create_table_titleAKAs = """CREATE TABLE IF NOT EXISTS titleAKAs(
                                akasID INT AUTO_INCREMENT PRIMARY KEY,
                                titleID VARCHAR(11),
                                ordering INT,
                                title TEXT,
                                region TEXT,
                                language TEXT,
                                isoriginaltitle BOOLEAN,
                                FOREIGN KEY (titleID) REFERENCES titlebasics(tconst))"""

    create_table_namebasics = """CREATE TABLE IF NOT EXISTS namebasics(
                                nconst VARCHAR(11) PRIMARY KEY,
                                primaryname TEXT,
                                birthyear DATE,
                                deathyear DATE,
                                noofmovies INT,
                                age INT,
                                currentdate DATE)"""
    
    create_table_principals = """CREATE TABLE IF NOT EXISTS principals(
                                principalsID INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(11),
                                ordering INT,
                                category TINYTEXT,
                                job TINYTEXT,
                                nconst VARCHAR(11),
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (nconst) REFERENCES namebasics(nconst))"""
    
    create_table_titlecrew = """CREATE TABLE IF NOT EXISTS titlecrew(
                                crewid INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(11) UNIQUE,
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""
    
    create_table_characters = """CREATE TABLE IF NOT EXISTS characters(
                                characterID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(256) UNIQUE)"""
    
    create_table_writers = """CREATE TABLE IF NOT EXISTS writers(
                                writerID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(256) UNIQUE)"""
    
    create_table_directors = """CREATE TABLE IF NOT EXISTS directors(
                                directorID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(256) UNIQUE)"""
    
    create_table_genres = """CREATE TABLE IF NOT EXISTS genres(
                                genreID INT AUTO_INCREMENT  PRIMARY KEY,
                                genre VARCHAR(256) UNIQUE)"""
    
    create_table_knownfortitles = """CREATE TABLE IF NOT EXISTS knownfortitles(
                                knownfortitlesID INT AUTO_INCREMENT PRIMARY KEY,
                                titles VARCHAR(11) UNIQUE)"""
    
    create_table_primaryprofession = """CREATE TABLE IF NOT EXISTS primaryprofession(
                                primaryprofessionID INT AUTO_INCREMENT PRIMARY KEY,
                                profession VARCHAR(256) UNIQUE)"""
    
    create_table_types = """CREATE TABLE IF NOT EXISTS types(
                                typeID INT AUTO_INCREMENT  PRIMARY KEY,
                                type VARCHAR(256) UNIQUE)"""
    
    create_table_attributes = """CREATE TABLE IF NOT EXISTS attributes(
                                attributeID INT AUTO_INCREMENT PRIMARY KEY,
                                attribute VARCHAR(256) UNIQUE)"""
    
    create_table_genretitlebasics = """CREATE TABLE IF NOT EXISTS genretitlebasics(
                                titlebasicsID VARCHAR(11),
                                genreID INT,
                                PRIMARY KEY (titlebasicsID, genreID),
                                FOREIGN KEY (titlebasicsID) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (genreID) REFERENCES genres(genreID))"""
    
    create_table_crewdirectors = """CREATE TABLE IF NOT EXISTS crewdirectors(
                                crewid INT,
                                directorid INT,
                                PRIMARY KEY (crewid ,directorid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (directorid) REFERENCES directors(directorID))"""
    
    create_table_crewwriters = """CREATE TABLE IF NOT EXISTS crewwriters(
                                crewid INT,
                                writerid INT,
                                PRIMARY KEY (crewid, writerid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (writerid) REFERENCES writers(writerid))"""
    
    create_table_principalscharacters = """CREATE TABLE IF NOT EXISTS principalscharacters(
                                principalsid INT,
                                characterid INT,
                                PRIMARY KEY (principalsid, characterid),
                                FOREIGN KEY (principalsid) REFERENCES principals(principalsID),
                                FOREIGN KEY (characterid) REFERENCES characters(characterid))"""
    
    create_table_AKAattributes = """CREATE TABLE IF NOT EXISTS AKAattributes(
                                akasid INT,
                                attributeid INT,
                                PRIMARY KEY (akasid, attributeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (attributeid) REFERENCES attributes(attributeid))"""
    
    create_table_AKAtype = """CREATE TABLE IF NOT EXISTS AKAtype(
                                akasid INT,
                                typeid INT,
                                PRIMARY KEY (akasid, typeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (typeid) REFERENCES types(typeid))"""
    
    create_table_primaryprofessiontitlenames = """CREATE TABLE IF NOT EXISTS primaryprofessiontitlenames(
                                primaryprofessionid INT,
                                namebasicsid VARCHAR(11),
                                PRIMARY KEY (primaryprofessionid, namebasicsid),
                                FOREIGN KEY (primaryprofessionid) REFERENCES primaryprofession(primaryprofessionid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst))"""
    
    create_table_knownfortitlenames = """CREATE TABLE IF NOT EXISTS knownfortitlenames(
                                namebasicsid VARCHAR(11),
                                knownfortitlesid INT,
                                PRIMARY KEY (namebasicsid, knownfortitlesid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst),
                                FOREIGN KEY (knownfortitlesid) REFERENCES knownfortitles(knownfortitlesid))"""
    
    conn = create_connection_nodb(host_name, user_name, user_password)
    
    # create database object if not already created and then connect the cursor to it
    if conn is not None:
        create_database(conn, database)
        conn = create_connection(host_name, user_name, user_password, database)
        
        # create tables
        if conn is not None:
            
            # create Title Basics table
            create_table(conn, create_table_titlebasics)

            # create Title Ratings table
            create_table(conn, create_table_titleratings)
            
            # create title episodes table
            create_table(conn, create_table_titleepisodes)
            
            # create title AKAs table
            create_table(conn, create_table_titleAKAs)
            
            # create name basics table
            create_table(conn, create_table_namebasics)
            
            # create principals table
            create_table(conn, create_table_principals)
            
            # create title crew table
            create_table(conn, create_table_titlecrew)
            
            # create characters table
            create_table(conn,create_table_characters)
            
            # create writers table
            create_table(conn,create_table_writers)
            
            # create directors table
            create_table(conn,create_table_directors)
            
            # create genres table
            create_table(conn,create_table_genres)
            
            # create known for titles table
            create_table(conn,create_table_knownfortitles)
            
            # create primary profession table
            create_table(conn,create_table_primaryprofession)
            
            # create type table
            create_table(conn,create_table_types)
            
            # create attributes table
            create_table(conn,create_table_attributes)
            
            # create genre title basics table
            create_table(conn, create_table_genretitlebasics)
            
            # create crew directors table
            create_table(conn,create_table_crewdirectors)
            
            # create crew writers table
            create_table(conn,create_table_crewwriters)
            
            # create principals characters table
            create_table(conn, create_table_principalscharacters)
            
            # create AKA Attributes table
            create_table(conn, create_table_AKAattributes)
            
            # create AKA Type table
            create_table(conn, create_table_AKAtype)
            
            # create Primary Profession Title Names table
            create_table(conn, create_table_primaryprofessiontitlenames)
            
            # create known for title names table
            create_table(conn, create_table_knownfortitlenames)
                
        else:
            print("Error! cannot create the database connection.")
    
    else:
        print("Error! Cannot create the MySQL connection.")
    
    


if __name__ == '__main__':
    main()


Connection to MySQL DB successful
Database created successfully
Connection to MySQL DB successful
Successfully created table: titlebasics
Successfully created table: titleratings
Successfully created table: titleepisodes
Successfully created table: titleAKAs
Successfully created table: namebasics
Successfully created table: principals
Successfully created table: titlecrew
Successfully created table: characters
Successfully created table: writers
Successfully created table: directors
Successfully created table: genres
Successfully created table: knownfortitles
Successfully created table: primaryprofession
Successfully created table: types
Successfully created table: attributes
Successfully created table: genretitlebasics
Successfully created table: crewdirectors
Successfully created table: crewwriters
Successfully created table: principalscharacters
Successfully created table: AKAattributes
1824 (HY000): Failed to open the referenced table 'type'
Successfully created table: primaryprofe

## SQL communication helper ##

In [467]:
import sqlalchemy
database_username = ''
database_password = ''
database_ip       = ''
database_name     = ''
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [15]:
## Helper Functions

In [411]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
    return 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Title Basics

In [8]:
# read title basic file
title_basics = pd.read_csv('/Users/abasiamaakpan/Downloads/title.basics.tsv', delimiter='\t',encoding='utf-8')
title_basics

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7292276,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7292277,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7292278,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7292279,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [9]:
def preprocess_dataframe_title_basics(data_frame):
    title_basics_new_series=data_frame

    b = pd.to_datetime(title_basics_new_series['runtimeMinutes'], errors='coerce', format='%M')

    b.to_frame().runtimeMinutes
    df_1 = pd.DataFrame(dict(time_stamps = b)) 

    df_1['runtimeMinutes'] = df_1['time_stamps'].dt.minute
    title_basics_new_series=title_basics_new_series.drop(['runtimeMinutes'],axis=1)
    title_basics_new_series['runtimeMinutes']=df_1['runtimeMinutes']
    
    title_basics_new_frame_transpose=title_basics_new_series.astype({'isAdult': 'int32'})
    
    title_basics_new_frame_transpose["isAdult"]=title_basics_new_frame_transpose.isAdult.mask(title_basics_new_frame_transpose.isAdult > 1,1)

    title_basics_new_frame_transpose['startYear']= pd.to_datetime(title_basics.startYear, errors='coerce', format='%Y')

    
    title_basics_new_frame_transpose['endYear']= pd.to_datetime(title_basics.endYear, errors='coerce', format='%Y')
    title_basics_new_frame_transpose=title_basics_new_frame_transpose.drop(['genres'],axis=1)
    new_row = {
        'tconst':None, 
        'titleType':None, 
        'primaryTitle':None,
        'originalTitle':None,
        'isAdult':None, 
        'startYear':None,
        'endYear':None,
        'runtimeMinutes':None         
    } 
  
    title_basics_new_frame_transpose = title_basics_new_frame_transpose.append(new_row, ignore_index=True)
    title_basics_new_frame_transpose_new=title_basics_new_frame_transpose.rename(columns={"runtimeMinutes": "runTimeMinutes"})

    return title_basics_new_frame_transpose_new

title_basics_preprocessed=preprocess_dataframe_title_basics(title_basics)

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [10]:
len(title_basics_preprocessed)
title_basics_preprocessed

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runTimeMinutes
0,tt0000001,short,Carmencita,Carmencita,0,1894-01-01,NaT,1.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892-01-01,NaT,5.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892-01-01,NaT,4.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892-01-01,NaT,12.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893-01-01,NaT,1.0
...,...,...,...,...,...,...,...,...
7292277,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010-01-01,NaT,NaN
7292278,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010-01-01,NaT,NaN
7292279,tt9916856,short,The Wind,The Wind,0,2015-01-01,NaT,27.0
7292280,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014-01-01,NaT,10.0


In [468]:
batch_size = 100000
length = len(title_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_basics_preprocessed, batch_size),total=length):
    load_dataframe(chunk,'titlebasics')

 99%|█████████▊| 72/73 [03:50<00:03,  3.21s/it]


IntegrityError: (mysql.connector.errors.IntegrityError) 1048 (23000): Column 'tconst' cannot be null
[SQL: INSERT INTO titlebasics (tconst, `titleType`, `primaryTitle`, `originalTitle`, `isAdult`, `startYear`, `endYear`, `runTimeMinutes`) VALUES (%(tconst)s, %(titleType)s, %(primaryTitle)s, %(originalTitle)s, %(isAdult)s, %(startYear)s, %(endYear)s, %(runTimeMinutes)s)]
[parameters: ({'tconst': 'tt9722998', 'titleType': 'short', 'primaryTitle': 'Gut gegessen', 'originalTitle': 'Gut gegessen', 'isAdult': 0, 'startYear': datetime.datetime(2015, 1, 1, 0, 0), 'endYear': None, 'runTimeMinutes': 32.0}, {'tconst': 'tt9723000', 'titleType': 'tvEpisode', 'primaryTitle': 'Glenn Close/Byron Bowers', 'originalTitle': 'Glenn Close/Byron Bowers', 'isAdult': 0, 'startYear': datetime.datetime(2019, 1, 1, 0, 0), 'endYear': None, 'runTimeMinutes': 40.0}, {'tconst': 'tt9723002', 'titleType': 'short', 'primaryTitle': 'Adagio', 'originalTitle': 'Adagio', 'isAdult': 0, 'startYear': datetime.datetime(2016, 1

## Title ratings ##

In [36]:
# read data from title ratings file
title_ratings = pd.read_csv('/Users/abasiamaakpan/Downloads/title.ratings.tsv', delimiter='\t',encoding='utf-8')
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1656
1,tt0000002,6.1,201
2,tt0000003,6.5,1369
3,tt0000004,6.2,122
4,tt0000005,6.2,2151
...,...,...,...
1086697,tt9916580,7.2,5
1086698,tt9916690,6.6,5
1086699,tt9916720,6.0,65
1086700,tt9916766,6.9,14


In [469]:
def preprocess_dataframe_title_ratings(data_frame):
    title_ratings_new_series=data_frame
    
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'averageRating': 'float'})
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'numVotes': 'int32'})
    #title_ratings_preprocessed.tconst = title_ratings_preprocessed.tconst.mask(~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)
    return title_ratings_new_frame_avg_rating
title_ratings_preprocessed=preprocess_dataframe_title_ratings(title_ratings)

In [458]:
len(title_ratings_preprocessed)

1086702

In [464]:
#title_ratings_preprocessed[~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst)]

title_ratings_preprocessed.tconst = title_ratings_preprocessed.tconst.mask(~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)



,tconst,averageRating,numVotes


In [470]:
batch_size = 100000
length = len(title_ratings_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_ratings_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleratings')

 91%|█████████ | 10/11 [00:40<00:04,  4.09s/it]


IntegrityError: (mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`last_trial_for_the_night`.`titleratings`, CONSTRAINT `titleratings_ibfk_1` FOREIGN KEY (`tconst`) REFERENCES `titlebasics` (`tconst`))
[SQL: INSERT INTO titleratings (tconst, `averageRating`, `numVotes`) VALUES (%(tconst)s, %(averageRating)s, %(numVotes)s)]
[parameters: ({'tconst': 'tt7348840', 'averageRating': 6.2, 'numVotes': 10}, {'tconst': 'tt7348842', 'averageRating': 6.1, 'numVotes': 10}, {'tconst': 'tt7348844', 'averageRating': 6.0, 'numVotes': 9}, {'tconst': 'tt7348952', 'averageRating': 6.9, 'numVotes': 9}, {'tconst': 'tt7348976', 'averageRating': 6.6, 'numVotes': 8}, {'tconst': 'tt7348978', 'averageRating': 6.8, 'numVotes': 6}, {'tconst': 'tt7349002', 'averageRating': 7.1, 'numVotes': 8}, {'tconst': 'tt7349004', 'averageRating': 6.8, 'numVotes': 6}  ... displaying 10 of 86702 total bound parameter sets ...  {'tconst': 'tt9916766', 'averageRa

## Title Episodes ##

In [42]:
# read title episodes data
title_episodes = pd.read_csv('/Users/abasiamaakpan/Downloads/title.episode.tsv', delimiter='\t',encoding='utf-8')
title_episodes

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
...,...,...,...,...
5252722,tt9916846,tt1289683,3,18
5252723,tt9916848,tt1289683,3,17
5252724,tt9916850,tt1289683,3,19
5252725,tt9916852,tt1289683,3,20


In [490]:
def preprocess_dataframe_title_episodes(data_frame):
    title_episodes_new_series=data_frame
    title_episodes_coerce_seasonNum = pd.to_numeric(title_episodes_new_series['seasonNumber'], errors='coerce')
    title_episodes_coerce_episodeNum = pd.to_numeric(title_episodes_new_series['episodeNumber'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_episodes_coerce_seasonNum)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_episodes_coerce_episodeNum)) 

    df_1['seasonNumber'] = df_1['test_column_1']
    df_2['episodeNumber'] = df_2['test_column_2']
    
    title_episodes_new_frame=title_episodes_new_series.drop(['seasonNumber'],axis=1)
    title_episodes_new_frame=title_episodes_new_series.drop(['episodeNumber'],axis=1)
    
    title_episodes_new_frame['seasonNumber']=df_1['seasonNumber']
    title_episodes_new_frame['episodeNumber']=df_2['episodeNumber']
    return title_episodes_preprocessed

title_episodes_preprocessed = preprocess_dataframe_title_episodes(title_episodes)
title_episodes_preprocessed

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0
...,...,...,...,...
5252722,tt9916846,tt1289683,3.0,18.0
5252723,tt9916848,tt1289683,3.0,17.0
5252724,tt9916850,tt1289683,3.0,19.0
5252725,tt9916852,tt1289683,3.0,20.0


In [491]:
title_episodes_preprocessed[~title_episodes_preprocessed.parentTconst.isin(title_basics_preprocessed.tconst)]



,tconst,parentTconst,seasonNumber,episodeNumber


In [492]:
title_episodes_preprocessed.tconst = title_episodes_preprocessed.tconst.mask(~title_episodes_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)



In [ ]:
title_episodes_preprocessed[~title_episodes_preprocessed.parentTconst.isin(title_basics_preprocessed.tconst)]



In [493]:
batch_size = 100000
length = len(title_episodes_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_episodes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleepisodes')

  0%|          | 0/53 [00:03<?, ?it/s]


IntegrityError: (mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`last_trial_for_the_night`.`titleepisodes`, CONSTRAINT `titleepisodes_ibfk_1` FOREIGN KEY (`parentTconst`) REFERENCES `titlebasics` (`tconst`))
[SQL: INSERT INTO titleepisodes (tconst, `parentTconst`, `seasonNumber`, `episodeNumber`) VALUES (%(tconst)s, %(parentTconst)s, %(seasonNumber)s, %(episodeNumber)s)]
[parameters: ({'tconst': 'tt0041951', 'parentTconst': 'tt0041038', 'seasonNumber': 1.0, 'episodeNumber': 9.0}, {'tconst': 'tt0042816', 'parentTconst': 'tt0989125', 'seasonNumber': 1.0, 'episodeNumber': 17.0}, {'tconst': 'tt0042889', 'parentTconst': 'tt0989125', 'seasonNumber': None, 'episodeNumber': None}, {'tconst': 'tt0043426', 'parentTconst': 'tt0040051', 'seasonNumber': 3.0, 'episodeNumber': 42.0}, {'tconst': 'tt0043631', 'parentTconst': 'tt0989125', 'seasonNumber': 2.0, 'episodeNumber': 16.0}, {'tconst': 'tt0043693', 'parentTconst': 'tt0989125

## Title Akas ##

In [47]:
title_akas = pd.read_csv('/Users/abasiamaakpan/Downloads/title.akas.tsv', delimiter='\t',encoding='utf-8')
title_akas

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
23944183,tt9916852,4,エピソード #3.20,JP,ja,\N,\N,0
23944184,tt9916852,5,Episódio #3.20,PT,pt,\N,\N,0
23944185,tt9916852,6,Episodio #3.20,IT,it,\N,\N,0
23944186,tt9916852,7,एपिसोड #3.20,IN,hi,\N,\N,0


In [494]:
def preprocess_dataframe_title_akas(data_frame):
    title_akas_new_series=data_frame
    title_akas_new_series=title_akas_new_series.drop(['types', 'attributes'],axis=1)
    
    title_akas_coerce_ordering = pd.to_numeric(title_akas_new_series['ordering'], errors='coerce')
    title_akas_coerce_isOriginalTitle = pd.to_numeric(title_akas_new_series['isOriginalTitle'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_akas_coerce_ordering)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_akas_coerce_isOriginalTitle)) 

    df_1['ordering'] = df_1['test_column_1']
    df_2['isOriginalTitle'] = df_2['test_column_2']
    
    title_akas_new_frame=title_akas_new_series.drop(['ordering'],axis=1)
    title_akas_new_frame=title_akas_new_series.drop(['isOriginalTitle'],axis=1)
    
    title_akas_new_frame['ordering']=df_1['ordering']
    title_akas_new_frame['isOriginalTitle']=df_2['isOriginalTitle']
   
    return title_akas_preprocessed
title_akas_preprocessed=preprocess_dataframe_title_akas(title_akas)

In [49]:
title_akas_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23944188 entries, 0 to 23944187
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   titleId          object 
 1   ordering         int64  
 2   title            object 
 3   region           object 
 4   language         object 
 5   isOriginalTitle  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ GB


In [495]:
title_akas_preprocessed[~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst)]

,titleId,ordering,title,region,language,isOriginalTitle
75529,tt0021006,1,"Ja, der Himmel über Wien",AT,\N,NaN
78082,tt0021453,1,Tapping Toes,US,\N,NaN
87197,tt0023019,1,Hollywood on Parade,US,\N,NaN
97581,tt0024677,1,Tom's in Town,US,\N,NaN
174103,tt0036165,1,Missing Men,US,\N,NaN
...,...,...,...,...,...,...
20551354,tt7978886,1,State of Defacto,AM,\N,0.0
20936004,tt8206494,1,Blood for Dust,US,\N,0.0
21374750,tt8466868,1,Butaca,ES,\N,0.0
22241328,tt8982514,1,To The Bitter End,US,\N,0.0


In [496]:
title_akas_preprocessed.titleId=title_akas_preprocessed.titleId.mask(~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst),None)

In [497]:
title_akas_preprocessed[~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst)]

,titleId,ordering,title,region,language,isOriginalTitle


In [498]:
batch_size=100000
length=len(title_akas_preprocessed)//batch_size+1
for chunk in tqdm(batch(title_akas_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleAKAs')

  1%|▏         | 3/240 [00:12<16:31,  4.18s/it]


DataError: (mysql.connector.errors.DataError) 1406 (22001): Data too long for column 'title' at row 20417
[SQL: INSERT INTO `titleAKAs` (`titleId`, ordering, title, region, language, `isOriginalTitle`) VALUES (%(titleId)s, %(ordering)s, %(title)s, %(region)s, %(language)s, %(isOriginalTitle)s)]
[parameters: ({'titleId': 'tt0050960', 'ordering': 4, 'title': 'En avant amiral!', 'region': 'FR', 'language': '\\N', 'isOriginalTitle': 0.0}, {'titleId': 'tt0050960', 'ordering': 5, 'title': 'The Ship Was Loaded', 'region': 'US', 'language': '\\N', 'isOriginalTitle': 0.0}, {'titleId': 'tt0050960', 'ordering': 6, 'title': 'Folytassa, admirális!', 'region': 'HU', 'language': '\\N', 'isOriginalTitle': 0.0}, {'titleId': 'tt0050960', 'ordering': 7, 'title': '¡A la orden, almirante!', 'region': 'ES', 'language': '\\N', 'isOriginalTitle': 0.0}, {'titleId': 'tt0050960', 'ordering': 8, 'title': 'Paina päälle, amiraali', 'region': 'FI', 'language': '\\N', 'isOriginalTitle': 0.0}, {'titleId': 'tt0050961',

## Name Basics

In [13]:
name_basics = pd.read_csv('/Users/abasiamaakpan/Downloads/name.basics.tsv', delimiter='\t',encoding='utf-8')
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0059956,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0072562,tt0080455,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0050976,tt0083922,tt0050986"
...,...,...,...,...,...,...
10464396,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10464397,nm9993716,Essias Loberg,\N,\N,NaN,\N
10464398,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10464399,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [14]:
def preprocess_dataframe_name_basics(data_frame):
    name_basics_new_series=data_frame
    name_basics_new_series=name_basics_new_series.drop(['primaryProfession', 'knownForTitles'],axis=1)
    
    name_basics_new_series['birthYear']= pd.to_datetime(name_basics_new_series.birthYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]
    
    name_basics_new_series['deathYear']= pd.to_datetime(name_basics_new_series.deathYear, errors='coerce', format='%Y')
    
    return name_basics_new_series

name_basics_preprocessed = preprocess_dataframe_name_basics(name_basics)

name_basics_preprocessed["noofmovies"] = ""
name_basics_preprocessed["age"] = ""
name_basics_preprocessed["currentdate"] = ""

In [15]:
name_basics_preprocessed_new= pd.to_numeric(name_basics_preprocessed['noofmovies'], errors='coerce')
name_basics_preprocessed_new = pd.to_numeric(name_basics_preprocessed['age'], errors='coerce')
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed.currentdate, errors='coerce', format='%Y')


In [16]:
df_1 = pd.DataFrame(dict(test_column_1 = name_basics_preprocessed_new)) 
df_2 = pd.DataFrame(dict(test_column_2 = name_basics_preprocessed_new)) 

df_1['noofmovies'] = df_1['test_column_1']
df_2['age'] = df_2['test_column_2']

name_basics_preprocessed['noofmovies']=df_1['noofmovies']
name_basics_preprocessed['age']=df_2['age']
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed['currentdate'])

In [17]:
len(name_basics_preprocessed)

10464401

In [18]:
name_basics_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10464401 entries, 0 to 10464400
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   nconst       object        
 1   primaryName  object        
 2   birthYear    datetime64[ns]
 3   deathYear    datetime64[ns]
 4   noofmovies   float64       
 5   age          float64       
 6   currentdate  datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(2)
memory usage: 558.9+ MB


## Approach 1- With sql alchemy ##

In [505]:
batch_size = 100000
length = len(name_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

100%|██████████| 105/105 [04:25<00:00,  2.53s/it]


In [57]:
# UNCOMMENT TO DISPLAY TABLE ROWS
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="alupizzo92",
#     database="imbd_application"
# )
# mycursor = mydb.cursor()

# query = "SELECT * FROM namebasics LIMIT 5;"
# mycursor.execute(query)

## Principals-will insert when nconst is in the right format  ##

In [3]:
title_principals = pd.read_csv('/Users/abasiamaakpan/Downloads/title.principals.tsv', delimiter='\t',encoding='utf-8')
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
41733807,tt9916880,5,nm0996406,director,principal director,\N
41733808,tt9916880,6,nm1482639,writer,\N,\N
41733809,tt9916880,7,nm2586970,writer,books,\N
41733810,tt9916880,8,nm1594058,producer,producer,\N


In [4]:
title_principals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41733812 entries, 0 to 41733811
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   tconst      object
 1   ordering    int64 
 2   nconst      object
 3   category    object
 4   job         object
 5   characters  object
dtypes: int64(1), object(5)
memory usage: 1.9+ GB


In [5]:
title_principal_new_series=title_principals.drop(['characters'],axis=1)



In [24]:
title_principal_new_series[~title_principal_new_series.nconst.isin(name_basics_preprocessed.nconst)]

,tconst,ordering,nconst,category,job
22073,tt0004069,7,None,actor,\N
25927,tt0004643,10,None,actress,\N
38369,tt0006395,7,None,actress,\N
39061,tt0006486,8,None,actor,\N
39364,tt0006539,9,None,actress,\N
...,...,...,...,...,...
41725564,tt9913366,7,None,self,\N
41727129,tt9913842,6,None,self,\N
41727662,tt9914050,6,None,self,\N
41728605,tt9914546,7,None,actor,\N


In [25]:
title_principal_new_series[~title_principal_new_series.tconst.isin(title_basics_preprocessed.tconst)]

,tconst,ordering,nconst,category,job


In [26]:
title_principal_new_series.nconst=title_principal_new_series.nconst.mask(~title_principal_new_series.nconst.isin(name_basics_preprocessed.nconst),None)
title_principal_new_series.tconst=title_principal_new_series.tconst.mask(~title_principal_new_series.tconst.isin(title_basics_preprocessed.tconst),None)


title_principal_new_series[~title_principal_new_series.nconst.isin(name_basics_preprocessed.nconst)]


,tconst,ordering,nconst,category,job
22073,tt0004069,7,None,actor,\N
25927,tt0004643,10,None,actress,\N
38369,tt0006395,7,None,actress,\N
39061,tt0006486,8,None,actor,\N
39364,tt0006539,9,None,actress,\N
...,...,...,...,...,...
41725564,tt9913366,7,None,self,\N
41727129,tt9913842,6,None,self,\N
41727662,tt9914050,6,None,self,\N
41728605,tt9914546,7,None,actor,\N


In [20]:
title_principal_new_series[~title_principal_new_series.tconst.isin(title_basics_preprocessed.tconst)]

,tconst,ordering,nconst,category,job


In [23]:
len(title_principal_new_series.tconst.unique())

6617142

In [ ]:
batch_size = 100000
length = len(title_principal_new_series)//batch_size+1

for chunk in tqdm(batch(title_principal_new_series,batch_size),total=length):
    load_dataframe(chunk,'principals')

 25%|██▌       | 105/418 [13:12<39:17,  7.53s/it]

## Title crew ##

In [27]:
title_crew = pd.read_csv('/Users/abasiamaakpan/Downloads/title.crew.tsv', delimiter='\t',encoding='utf-8')
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7292276,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7292277,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7292278,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7292279,tt9916856,nm10538645,nm6951431


In [28]:
title_crew_processed=title_crew.drop(['directors'],axis=1)
title_crew_processed=title_crew.drop(['writers'],axis=1)

In [ ]:
batch_size=100000
length=len(title_crew_processed)//batch_size+1

for chunk in tqdm(batch(title_crew_processed,batch_size),total=length):
    load_dataframe(chunk,'titlecrew')

## Writers ##

In [29]:
def preprocess_dataframe_parsing(data_frame, column_name, final_column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    data_preprocessed_unique= pd.DataFrame(data_frame.name.unique())
    data_preprocessed_unique_column_rename=data_preprocessed_unique.rename(columns={0:final_column_name})
    
    return data_preprocessed_unique_column_rename

#writers_preprocessed_new=writers_preprocessed_new.drop(['primaryProfession'],axis=1)



In [ ]:
writers_preprocessed_final=preprocess_dataframe_parsing(title_crew.writers, "writers", "name")


writers_preprocessed_final

In [ ]:
batch_size=100000
length=len(writers_preprocessed_final)//batch_size+1
for chunk in tqdm(batch(writers_preprocessed_final,batch_size),total=length):
    load_dataframe(chunk,'writers')

## Directors ##

In [30]:
directors_preprocessed=preprocess_dataframe_parsing(title_crew.directors, "directors", "name")

In [31]:
directors_preprocessed

,name
0,nm0005690
1,nm0721526
2,nm0374658
3,nm0085156
4,nm0525910
...,...
641677,nm10529675
641678,nm10439323
641679,nm10538612
641680,nm10538639


In [ ]:
batch_size=100000
length=len(directors_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'directors')

## Genres ##

In [32]:
genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres", "genre")

In [284]:
genres_preprocessed

,genres
0,Documentary
1,Short
2,Animation
3,Comedy
4,Romance
5,Sport
6,Action
7,News
8,Drama
9,Fantasy


In [75]:
len(genres_preprocessed)

11583271

In [ ]:
batch_size=100000
length=len(genres_preprocessed)//batch_size+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

## Known for titles ##

In [285]:
knownForTitles_preprocessed=preprocess_dataframe_parsing(name_basics.knownForTitles, "knownForTitles", "titles")

In [286]:
knownForTitles_preprocessed

,knownForTitles
0,tt0031983
1,tt0072308
2,tt0053137
3,tt0050419
4,tt0037382
...,...
1583516,tt8743558
1583517,tt8743742
1583518,tt8187876
1583519,tt8744074


In [ ]:
batch_size=100000
length=len(knownForTitles_preprocessed)//batch_size+1
for chunk in tqdm(batch(knownForTitles_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'knownFortitles')

## primary profession ##

In [287]:
pprofession_preprocessed=preprocess_dataframe_parsing(name_basics.primaryProfession, "primaryProfession", "profession")

In [288]:
pprofession_preprocessed

,primaryProfession
0,soundtrack
1,actor
2,miscellaneous
3,actress
4,music_department
5,writer
6,director
7,producer
8,make_up_department
9,composer


In [ ]:
batch_size=100000
length=len(pprofession_preprocessed)//batch_size+1
for chunk in tqdm(batch(pprofession_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'primaryprofession')

## types ## 

In [289]:
type_preprocessed=preprocess_dataframe_parsing(title_basics.titleType, "titleType", "type")

In [290]:
type_preprocessed

,titleType
0,short
1,movie
2,tvShort
3,tvMovie
4,tvSeries
5,tvEpisode
6,tvMiniSeries
7,tvSpecial
8,video
9,videoGame


In [141]:
batch_size=100000
length=len(type_preprocessed)//batch_size+1
for chunk in tqdm(batch(type_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'types')

100%|██████████| 240/240 [04:13<00:00,  1.06s/it]


## attributes ##

In [302]:
attributes_preprocessed=preprocess_dataframe_parsing(title_akas.attributes, "attributes", "attribute")

In [301]:
attributes_preprocessed

,attributes
0,\N
1,literal title
2,literal English title
3,informal alternative title
4,promotional title
...,...
184,first segment title
185,second segment title
186,fifth part title
187,incorrect title


In [84]:
batch_size=100000
length=len(attributes_preprocessed)//batch_size+1
for chunk in tqdm(batch(attributes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'attributes')

100%|██████████| 240/240 [04:37<00:00,  1.16s/it]


batch_size=100000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'attributes')

## Characters ##

In [421]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.replace('[\[\]"]',"").str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
characters_preprocessed=preprocess_dataframe_parsing(title_principals.characters, "characters")

In [422]:
characters_preprocessed_unique= pd.DataFrame(characters_preprocessed.name.unique())



In [424]:
characters_preprocessed_unique

,0
0,Self
1,\N
2,Blacksmith
3,Assistant
4,Sneezing Man
...,...
2262474,Self - The Nun
2262475,Nakesha
2262476,Tomeka
2262477,Alasia


In [ ]:
batch_size=100000
length=len(characters_preprocessed_unique)//batch_size+1
for chunk in tqdm(batch(characters_preprocessed_unique,batch_size),total=length):
    load_dataframe(chunk,'characters')

## Genre Title Basics-below functions work for the rest of the tables. Havent refactored all the tables yet

In [33]:
def createAuxilliaryTables(Frame1,Frame2, column_rename_1, column_rename_2):
    
    Frame3 = pd.merge(Frame1, Frame2, on=Frame1.index, how='outer')
    Frame4=Frame3.rename(columns={"key_0": column_rename_1})
    created_table = pd.merge(Frame4, Frame2, on=Frame4.index, how='outer')
    created_table=created_table.rename(columns={"key_0": column_rename_2})
        
    return created_table

In [34]:
def concat_tables (Frame1,Frame2,column1_name,column2_name,column1_name_new,column2_name_new):
    NewFrame_column1=createAuxilliaryTables(Frame1, Frame1, column1_name, column1_name_new)
    NewFrame_column2=createAuxilliaryTables(Frame2, Frame2, column2_name, column2_name_new)
    NewFrame_column2=NewFrame_column2.drop([column2_name_new],axis=1)
    NewFrame_column1=NewFrame_column1.drop([column1_name_new],axis=1)
    NewFrame_final_table = pd.concat([NewFrame_column2, NewFrame_column1], ignore_index=True)
    
    return NewFrame_final_table


In [35]:
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7292276,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7292277,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7292278,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7292279,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [40]:
title_basics_preprocessed_new_1=title_basics.drop(['tconst'],axis=1)
title_basics_preprocessed_new_2=title_basics_preprocessed_new_1.drop(['titleType'],axis=1)
title_basics_preprocessed_new_3=title_basics_preprocessed_new_2.drop(['primaryTitle'],axis=1)
title_basics_preprocessed_new_4=title_basics_preprocessed_new_3.drop(['originalTitle'],axis=1)
title_basics_preprocessed_new_5=title_basics_preprocessed_new_4.drop(['isAdult'],axis=1)
title_basics_preprocessed_new_6=title_basics_preprocessed_new_5.drop(['startYear'],axis=1)
title_basics_preprocessed_new_7=title_basics_preprocessed_new_6.drop(['endYear'],axis=1)
title_basics_preprocessed_new_8=title_basics_preprocessed_new_7.drop(['runtimeMinutes'],axis=1)
title_basics_preprocessed_new_9=title_basics_preprocessed_new_8.drop(['genres'],axis=1)



title_basics_preprocessed_new_9

""
0
1
2
3
4
...
7292276
7292277
7292278
7292279


In [41]:
genres_preprocessed

,genre
0,Documentary
1,Short
2,Animation
3,Comedy
4,Romance
5,Sport
6,Action
7,News
8,Drama
9,Fantasy


In [43]:
genres_preprocessed_new_1=genres_preprocessed.drop(['genre'],axis=1)

genres_preprocessed_new_1

""
0
1
2
3
4
5
6
7
8
9


In [45]:
genre_title_basics_final_table=concat_tables(title_basics_preprocessed_new_9,genres_preprocessed_new_1,"titilebasicsID","genreID","titlebasicsNew", "genreNew")

genre_title_basics_final_table

,genreID,titilebasicsID
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
7292306,NaN,7292276.0
7292307,NaN,7292277.0
7292308,NaN,7292278.0
7292309,NaN,7292279.0


In [ ]:
batch_size=100000
length=len(genre_title_basics_final_table)//batch_size+1
for chunk in tqdm(batch(genre_title_basics_final_table,batch_size),total=length):
    load_dataframe(chunk,'genretitlebasics')

## CrewDirectors

In [48]:
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7292276,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7292277,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7292278,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7292279,tt9916856,nm10538645,nm6951431


In [47]:
directors_preprocessed

,name
0,nm0005690
1,nm0721526
2,nm0374658
3,nm0085156
4,nm0525910
...,...
641677,nm10529675
641678,nm10439323
641679,nm10538612
641680,nm10538639


In [49]:
crew_director_preprocessed_new_1=title_crew.drop(['directors'],axis=1)
crew_director_preprocessed_new_2=crew_director_preprocessed_new_1.drop(['writers'],axis=1)
crew_director_preprocessed_new_3=crew_director_preprocessed_new_2.drop(['tconst'],axis=1)


directors_preprocessed_new_1=directors_preprocessed.drop(['name'],axis=1)

crew_director_preprocess_final_table=concat_tables(crew_director_preprocessed_new_3,directors_preprocessed_new_1,"crewid","directorid","crewNew", "directorNew")

#crew_director_preprocess_final_table

#crew_director_preprocess_column1=createAuxilliaryTables(crew_director_preprocessed_new_3, crew_director_preprocessed_new_3, "crewid", "crewNew")
#crew_director_preprocess_column2=createAuxilliaryTables(directors_preprocessed_new_1, directors_preprocessed_new_1, "directorId", "directorNew")


#crew_director_preprocess_column2=crew_director_preprocess_column2.drop(['directorNew'],axis=1)
##crew_director_preprocess_column1=crew_director_preprocess_column1.drop(['crewNew'],axis=1)
#crew_director_preprocess_final_table = pd.concat([crew_director_preprocess_column2, crew_director_preprocess_column1], ignore_index=True)

In [50]:
crew_director_preprocess_final_table

,directorid,crewid
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
7933958,NaN,7292276.0
7933959,NaN,7292277.0
7933960,NaN,7292278.0
7933961,NaN,7292279.0


In [ ]:
batch_size=100000
length=len(crew_director_preprocess_final_table)//batch_size+1
for chunk in tqdm(batch(crew_director_preprocess_final_table,batch_size),total=length):
    load_dataframe(chunk,'crewdirectors')

## Crew Writers

In [416]:
writers_preprocessed_final

,name
0,\N
1,nm0085156
2,nm0410331
3,nm0617588
4,nm0005717
...,...
809831,nm10538639
809832,nm10538642
809833,nm9641593
809834,nm10538643


In [ ]:
directors_preprocessed_new_1=directors_preprocessed.drop(['directors'],axis=1)

crew_director_preprocess_column1=createAuxilliaryTables(crew_director_preprocessed_new_3, crew_director_preprocessed_new_3, "crewid", "crewNew")
crew_director_preprocess_column2=createAuxilliaryTables(directors_preprocessed_new_1, directors_preprocessed_new_1, "directorId", "directorNew")

crew_director_preprocess_column2=crew_director_preprocess_column2.drop(['directorNew'],axis=1)
crew_director_preprocess_column1=crew_director_preprocess_column1.drop(['crewNew'],axis=1)
crew_director_preprocess_final_table = pd.concat([crew_director_preprocess_column2, crew_director_preprocess_column1], ignore_index=True)

In [ ]:
batch_size=100000
length=len(crew_writer_preprocess_final_table)//batch_size+1
for chunk in tqdm(batch(crew_writer_preprocess_final_table,batch_size),total=length):
    load_dataframe(chunk,'crewdirectors')

## Principal Characters

In [417]:
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
41733807,tt9916880,5,nm0996406,director,principal director,\N
41733808,tt9916880,6,nm1482639,writer,\N,\N
41733809,tt9916880,7,nm2586970,writer,books,\N
41733810,tt9916880,8,nm1594058,producer,producer,\N


In [442]:
principals_characters_preprocessed_1=title_principals.drop(['tconst'],axis=1)
principals_characters_preprocessed_2=principals_characters_preprocessed_1.drop(['ordering'],axis=1)
principals_characters_preprocessed_3=principals_characters_preprocessed_2.drop(['nconst'],axis=1)
principals_characters_preprocessed_4=principals_characters_preprocessed_3.drop(['category'],axis=1)
principals_characters_preprocessed_5=principals_characters_preprocessed_4.drop(['job'],axis=1)
principals_characters_preprocessed_6=principals_characters_preprocessed_5.drop(['characters'],axis=1)
principals_characters_preprocessed_6

""
0
1
2
3
4
...
41733807
41733808
41733809
41733810


In [441]:
characters_preprocessed_unique_new=characters_preprocessed_unique.drop(0,axis=1)
characters_preprocessed_unique_new

""
0
1
2
3
4
...
2262474
2262475
2262476
2262477


In [443]:
principal_character_final_table=concat_tables(principals_characters_preprocessed_6,characters_preprocessed_unique_new,"principalsid","characterid","principalsNew", "charactersNew")


In [444]:
principal_character_final_table

,characterid,principalsid
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
43996286,NaN,41733807.0
43996287,NaN,41733808.0
43996288,NaN,41733809.0
43996289,NaN,41733810.0


In [ ]:
batch_size=100000
length=len(principal_character_final_table)//batch_size+1
for chunk in tqdm(batch(principal_character_final_table,batch_size),total=length):
    load_dataframe(chunk,'principalscharacter')

## AKA Attributes-Create function maybe

In [155]:
title_akas_preprocessed_new_1=title_akas_preprocessed.drop(['titleId'],axis=1)
title_akas_preprocessed_new_2=title_akas_preprocessed_new_1.drop(['ordering'],axis=1)
title_akas_preprocessed_new_3=title_akas_preprocessed_new_2.drop(['title'],axis=1)
title_akas_preprocessed_new_4=title_akas_preprocessed_new_3.drop(['region'],axis=1)
title_akas_preprocessed_new_5=title_akas_preprocessed_new_4.drop(['language'],axis=1)
title_akas_preprocessed_new_6=title_akas_preprocessed_new_5.drop(['isOriginalTitle'],axis=1)
title_akas_preprocessed_new_6

In [368]:
title_akas_preprocessed

,titleId,ordering,title,region,language,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,0.0
1,tt0000001,2,Carmencita,DE,\N,0.0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,0.0
3,tt0000001,4,Καρμενσίτα,GR,\N,0.0
4,tt0000001,5,Карменсита,RU,\N,0.0
...,...,...,...,...,...,...
23944183,tt9916852,4,エピソード #3.20,JP,ja,0.0
23944184,tt9916852,5,Episódio #3.20,PT,pt,0.0
23944185,tt9916852,6,Episodio #3.20,IT,it,0.0
23944186,tt9916852,7,एपिसोड #3.20,IN,hi,0.0


In [299]:
title_akas_preprocessed_new_1=title_akas_preprocessed.drop(['titleId'],axis=1)
title_akas_preprocessed_new_2=title_akas_preprocessed_new_1.drop(['ordering'],axis=1)
title_akas_preprocessed_new_3=title_akas_preprocessed_new_2.drop(['title'],axis=1)
title_akas_preprocessed_new_4=title_akas_preprocessed_new_3.drop(['region'],axis=1)
title_akas_preprocessed_new_5=title_akas_preprocessed_new_4.drop(['language'],axis=1)
title_akas_preprocessed_new_6=title_akas_preprocessed_new_5.drop(['isOriginalTitle'],axis=1)
title_akas_preprocessed_new_6

""
0
1
2
3
4
...
23944183
23944184
23944185
23944186


In [314]:
attributes_preprocessed

,attribute
0,\N
1,literal title
2,literal English title
3,informal alternative title
4,promotional title
...,...
184,first segment title
185,second segment title
186,fifth part title
187,incorrect title


In [310]:
attributes_preprocessed_new_1=attributes_preprocessed.drop(['attribute'],axis=1)



In [315]:
attributes_preprocessed_new_1

""
0
1
2
3
4
...
184
185
186
187


In [304]:
aka_attribute_preprocess_column1=createAuxilliaryTables(title_akas_preprocessed_new_6, title_akas_preprocessed_new_6, "akasid", "akasidNew")



In [305]:
aka_attribute_preprocess_column2=createAuxilliaryTables(attributes_preprocessed_new_1, attributes_preprocessed_new_1, "attributeid", "attributeidNew")



In [306]:
aka_attribute_preprocess_column2=aka_attribute_preprocess_column2.drop(['attributeidNew'],axis=1)
aka_attribute_preprocess_column1=aka_attribute_preprocess_column1.drop(['akasidNew'],axis=1)

In [307]:
aka_attribute_final_table = pd.concat([aka_attribute_preprocess_column2, aka_attribute_preprocess_column1], ignore_index=True)

In [308]:
aka_attribute_final_table

,attributeid,akasid
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
47888371,NaN,23944183.0
47888372,NaN,23944184.0
47888373,NaN,23944185.0
47888374,NaN,23944186.0


In [134]:
batch_size=100000
length=len(aka_attribute_final_table)//batch_size+1
for chunk in tqdm(batch(aka_attribute_final_table,batch_size),total=length):
    load_dataframe(chunk,'AKAattributes')

100%|██████████| 240/240 [04:24<00:00,  1.10s/it]


## AKA Type- Use function

In [316]:
title_akas_preprocessed_new_6

""
0
1
2
3
4
...
23944183
23944184
23944185
23944186


In [317]:
type_preprocessed

,titleType
0,short
1,movie
2,tvShort
3,tvMovie
4,tvSeries
5,tvEpisode
6,tvMiniSeries
7,tvSpecial
8,video
9,videoGame


In [325]:
type_preprocess_column1=createAuxilliaryTables(title_akas_preprocessed_new_6, title_akas_preprocessed_new_6, "akasid", "akasidNew")
type_preprocess_column1

,akasidNew,akasid
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
23944183,23944183,23944183
23944184,23944184,23944184
23944185,23944185,23944185
23944186,23944186,23944186


In [326]:
type_preprocess_column2=createAuxilliaryTables(type_preprocessed, type_preprocessed, "typeid", "typeidNew")
type_preprocess_column2

,typeidNew,typeid,titleType_x,titleType_y,titleType
0,0,0,short,short,short
1,1,1,movie,movie,movie
2,2,2,tvShort,tvShort,tvShort
3,3,3,tvMovie,tvMovie,tvMovie
4,4,4,tvSeries,tvSeries,tvSeries
5,5,5,tvEpisode,tvEpisode,tvEpisode
6,6,6,tvMiniSeries,tvMiniSeries,tvMiniSeries
7,7,7,tvSpecial,tvSpecial,tvSpecial
8,8,8,video,video,video
9,9,9,videoGame,videoGame,videoGame


In [327]:
type_preprocess_column3=type_preprocess_column2.drop(['typeidNew'],axis=1)
type_preprocess_column4=type_preprocess_column3.drop(['titleType_x'],axis=1)
type_preprocess_column5=type_preprocess_column4.drop(['titleType_y'],axis=1)
type_preprocess_column6=type_preprocess_column5.drop(['titleType'],axis=1)
type_preprocess_column1=type_preprocess_column1.drop(['akasidNew'],axis=1)
type_preprocess_column6


,typeid
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [328]:
type_preprocess_column1

,akasid
0,0
1,1
2,2
3,3
4,4
...,...
23944183,23944183
23944184,23944184
23944185,23944185
23944186,23944186


In [330]:
type_final_table = pd.concat([type_preprocess_column1, type_preprocess_column6], ignore_index=True)

In [331]:
type_final_table

,akasid,typeid
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
23944193,NaN,5.0
23944194,NaN,6.0
23944195,NaN,7.0
23944196,NaN,8.0


In [ ]:
batch_size=100000
length=len(aka_attribute_final_table)//batch_size+1
for chunk in tqdm(batch(aka_attribute_final_table,batch_size),total=length):
    load_dataframe(chunk,'AKAtype')

## Primaryprofessiontitlenames

In [355]:
pprofession_preprocessed

,primaryProfession
0,soundtrack
1,actor
2,miscellaneous
3,actress
4,music_department
5,writer
6,director
7,producer
8,make_up_department
9,composer


In [374]:
primaryprofessiontitlenames= pd.DataFrame(pprofession_preprocessed)
primaryprofessiontitlenames

,primaryProfession
0,soundtrack
1,actor
2,miscellaneous
3,actress
4,music_department
5,writer
6,director
7,producer
8,make_up_department
9,composer


In [377]:
primaryprofessiontitlenames_new_unique=primaryprofessiontitlenames.drop("primaryProfession",axis=1)

primaryprofessiontitlenames_new_unique

""
0
1
2
3
4
5
6
7
8
9


In [378]:
name_basics_preprocessed_new=name_basics_preprocessed.drop("nconst",axis=1)
#name_basics_preprocessed_new=name_basics_preprocessed_new.drop("primaryName",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("birthYear",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("deathYear",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("noofmovies",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("age",axis=1)
name_basics_preprocessed_new_unique=name_basics_preprocessed_new.drop("currentdate",axis=1)
#name_basics_preprocessed_new_unique= pd.DataFrame(name_basics_preprocessed_new.primaryName.unique())



In [379]:
name_basics_preprocessed_new_unique=name_basics_preprocessed_new_unique.rename(columns={0: "primaryName"})
name_basics_preprocessed_new_unique

,primaryName
0,Fred Astaire
1,Lauren Bacall
2,Brigitte Bardot
3,John Belushi
4,Ingmar Bergman
...,...
10464396,Romeo del Rosario
10464397,Essias Loberg
10464398,Harikrishnan Rajan
10464399,Aayush Nair


In [380]:
name_basics_preprocessed_unique_new=name_basics_preprocessed_new_unique.drop("primaryName",axis=1)
name_basics_preprocessed_unique_new

""
0
1
2
3
4
...
10464396
10464397
10464398
10464399


In [381]:
name_basics_preprocess_column1=createAuxilliaryTables(primaryprofessiontitlenames_new_unique, primaryprofessiontitlenames_new_unique, "primaryprofessionId", "primaryprofessionIdNew")
name_basics_preprocess_column1

,primaryprofessionIdNew,primaryprofessionId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9


In [382]:
name_basics_preprocess_column2=createAuxilliaryTables(name_basics_preprocessed_unique_new, name_basics_preprocessed_unique_new, "namebasicsId", "namebasicsIdNew")
name_basics_preprocess_column2

,namebasicsIdNew,namebasicsId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
10464396,10464396,10464396
10464397,10464397,10464397
10464398,10464398,10464398
10464399,10464399,10464399


In [383]:
name_basics_preprocess_column2=name_basics_preprocess_column2.drop(['namebasicsIdNew'],axis=1)
name_basics_preprocess_column1=name_basics_preprocess_column1.drop(['primaryprofessionIdNew'],axis=1)



In [384]:
primaryprofession_final_table = pd.concat([name_basics_preprocess_column2, name_basics_preprocess_column1], ignore_index=True)

In [391]:
primaryprofession_final_table.tail()

,namebasicsId,primaryprofessionId
10464437,NaN,36.0
10464438,NaN,37.0
10464439,NaN,38.0
10464440,NaN,39.0
10464441,NaN,40.0


In [ ]:
batch_size=100000
length=len(primaryprofession_final_table)//batch_size+1
for chunk in tqdm(batch(primaryprofession_final_table),total=length):
    load_dataframe(chunk,'primaryprofessiontitlenames')

## Known for Title Names

In [354]:
knownForTitles_preprocessed

,knownForTitles
0,tt0031983
1,tt0072308
2,tt0053137
3,tt0050419
4,tt0037382
...,...
1583516,tt8743558
1583517,tt8743742
1583518,tt8187876
1583519,tt8744074


In [388]:
knownForTitles_preprocessed_new=knownForTitles_preprocessed.drop(['knownForTitles'],axis=1)



In [389]:
knownForTitles_preprocessed_new

""
0
1
2
3
4
...
1583516
1583517
1583518
1583519


In [366]:
name_basics_preprocessed_unique_new

""
0
1
2
3
4
...
10464396
10464397
10464398
10464399


In [395]:
knownForTitleNames_preprocess_column1=createAuxilliaryTables(knownForTitles_preprocessed_new, knownForTitles_preprocessed_new, "knownForTitlesId", "knownForTitlesNew")

knownForTitleNames_preprocess_column2=createAuxilliaryTables(name_basics_preprocessed_unique_new, name_basics_preprocessed_unique_new, "namebasicsId", "namebasicsNew")

knownForTitleNames_preprocess_column1

,knownForTitlesNew,knownForTitlesId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
1583516,1583516,1583516
1583517,1583517,1583517
1583518,1583518,1583518
1583519,1583519,1583519


In [396]:
knownForTitleNames_preprocess_column2=knownForTitleNames_preprocess_column2.drop(['namebasicsNew'],axis=1)
knownForTitleNames_preprocess_column1=knownForTitleNames_preprocess_column1.drop(['knownForTitlesNew'],axis=1)
knownForTitleNames_final_table = pd.concat([knownForTitleNames_preprocess_column2, knownForTitleNames_preprocess_column1], ignore_index=True)
knownForTitleNames_final_table

,namebasicsId,knownForTitlesId
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
12047917,NaN,1583516.0
12047918,NaN,1583517.0
12047919,NaN,1583518.0
12047920,NaN,1583519.0


In [ ]:
batch_size=100000
length=len(knownForTitleNames_final_table)//batch_size+1
for chunk in tqdm(batch(knownForTitleNames_final_table,batch_size),total=length):
    load_dataframe(chunk,'knownfortitlenames')

## Question 5

In [ ]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

c.execute("DROP TABLE imbd_application")


In [ ]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

operation = """SELECT COUNT(KTN.knownfortitlesID)
               FROM namebasics AS NB, knownfortitlenames AS KTN
               WHERE KTN.knownfortitlesID IN (SELECT KT.knownfortitlesID 
                                              FROM knownforTitles AS KT 
                                              WHERE KT.titles IN (SELECT tconst
                                                                  FROM titlebasics AS TB 
                                                                  WHERE TB.titletype = 'Movie'))
                GROUP BY namebasicsID
                LIMIT 5"""

for result in c.execute(operation, multi=True):
    if result.with_rows:
        print("Rows produced by statement '{}':".format(
          result.statement))
        print(result.fetchall())
    else:
        print("Number of rows affected by statement '{}': {}".format(
          result.statement, result.rowcount))